# Limpieza de datos

### Importo librerias

In [1]:
import re
import difflib
import numpy as np
import pandas as pd

### Cargo el excel con pandas

In [2]:
full_data = pd.read_excel('./Vecindarios_Alimentarios_Encuesta_Integrada_2022_11_23.xlsx', sheet_name=None)

Obtengo todos los nombres de las hojas del excel

In [3]:
for name in full_data:
    print(name)

Vecindarios Alimentarios - E...
composicion_hogar
_5_18_medidas_control_plagas
_5_19_productos_control_plagas
_5_20_control_malezas
_7_1_Oferta_Alimentos_Produccio
_7_2_produc_dejo_producir_vend
Productos_Vende_Canal
canales_venta_productores
aplicacion_biopreparados
personas_trabajan_puesto_venta
canales_compra_comerciantes
Productos_Compra_Canal
APOYO_RECIBIDO
APOYO_BRINDADO
HUERTO_RELACIONES


dropeo las columnas no relevanntes para este trabajo de limpieza de datos

In [4]:
columns_to_drop = ['_submission__id', '_submission__uuid', '_submission__submission_time', '_submission__validation_status', '_submission__notes', '_submission__status', '_submission__submitted_by', '_submission__tags']

In [5]:
vecindarios = full_data['Vecindarios Alimentarios - E...']
composicion_hogar = full_data['composicion_hogar'].drop(columns=[*columns_to_drop, '_index'])
medidas_control_plagas = full_data['_5_18_medidas_control_plagas'].drop(columns=[*columns_to_drop, '_index'])
productos_control_plagas = full_data['_5_19_productos_control_plagas'].drop(columns=[*columns_to_drop, '_index'])
control_malezas = full_data['_5_20_control_malezas'].drop(columns=[*columns_to_drop, '_index'])
oferta_alimentos = full_data['_7_1_Oferta_Alimentos_Produccio'].drop(columns=[*columns_to_drop, '_index'])
producto_dejo_producir = full_data['_7_2_produc_dejo_producir_vend'].drop(columns=[*columns_to_drop, '_index'])
producto_vende_canal = full_data['Productos_Vende_Canal'].drop(columns=[*columns_to_drop])
canales_venta_productores = full_data['canales_venta_productores'].drop(columns=[*columns_to_drop])
aplicacion_biopreparados = full_data['aplicacion_biopreparados'].drop(columns=[*columns_to_drop, '_index'])
personas_trabajan_puesto_venta = full_data['personas_trabajan_puesto_venta'].drop(columns=[*columns_to_drop, '_index'])
canales_compra_comerciantes = full_data['canales_compra_comerciantes'].drop(columns=[*columns_to_drop])
productos_compra_canal = full_data['Productos_Compra_Canal'].drop(columns=[*columns_to_drop])
APOYO_RECIBIDO = full_data['APOYO_RECIBIDO'].drop(columns=[*columns_to_drop])
APOYO_BRINDADO = full_data['APOYO_BRINDADO'].drop(columns=[*columns_to_drop])
HUERTO_RELACIONES = full_data['HUERTO_RELACIONES'].drop(columns=[*columns_to_drop])

### Composicion_hogar

In [6]:
df_ = composicion_hogar.loc[composicion_hogar['miembro_parentesco']=='el_entrevistado'][['_parent_index','sexo','edad_miembro','miembros_aporta_economicamente']].reset_index(drop=True)

In [7]:
df__ = vecindarios.loc[:,['_index','sexo','edad','aporta_economicamente_hogar']].reset_index(drop=True)

In [8]:
df__['sexo'] = df__.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['sexo'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['sexo'].size != 0 else np.NAN, axis=1)
df__['edad'] = df__.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['edad_miembro'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['edad_miembro'].size != 0 else np.NAN, axis=1)
df__['aporta_economicamente_hogar'] = df__.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['miembros_aporta_economicamente'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['miembros_aporta_economicamente'].size != 0 else np.NAN, axis=1)

In [9]:
vecindarios[['sexo','edad','aporta_economicamente_hogar']] = df__[['sexo','edad','aporta_economicamente_hogar']]

In [10]:
df_ = composicion_hogar.groupby(by=['_parent_index','sexo'], as_index=False).count()[['_parent_index','sexo','miembro_parentesco','otro_pariente','otro_NO_pariente']]
df_['suma'] = df_['miembro_parentesco'] + df_['otro_pariente'] + df_['otro_NO_pariente']

In [11]:
vecindarios['num_mujeres_hogar'] = vecindarios.apply(lambda x: df_.loc[(df_['_parent_index']==x['_index']) & (df_['sexo']=='femenino')]['suma'].iloc[0] if df_.loc[(df_['_parent_index']==x['_index']) & (df_['sexo']=='femenino')]['suma'].size != 0 else 0, axis=1)
vecindarios['num_hombres_hogar'] = vecindarios.apply(lambda x: df_.loc[(df_['_parent_index']==x['_index']) & (df_['sexo']=='masculino')]['suma'].iloc[0] if df_.loc[(df_['_parent_index']==x['_index']) & (df_['sexo']=='masculino')]['suma'].size != 0 else 0, axis=1)

In [12]:
df_ = composicion_hogar.loc[composicion_hogar['edad_miembro'] < 18].groupby(by=['_parent_index'], as_index=False).count()

In [13]:
vecindarios['num_menores_hogar'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['edad_miembro'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['edad_miembro'].size != 0 else 0, axis=1)

In [14]:
df_ = composicion_hogar.loc[composicion_hogar['miembros_aporta_economicamente']=='si'].groupby(by=['_parent_index'], as_index=False).count().rename(columns={'miembros_aporta_economicamente':'aporta_economicamente'})

In [15]:
vecindarios['num_miembros_aportan_hogar'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['aporta_economicamente'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['aporta_economicamente'].size != 0 else 0, axis=1)

### medidas_control_plagas

In [16]:
df_ = medidas_control_plagas.copy()

In [17]:
df_['_5_18_medida_control_plagas'] = df_['_5_18_medida_control_plagas'].str.lower().str.split(',')
df_ = df_.explode('_5_18_medida_control_plagas')
df_['_5_18_medida_control_plagas'] = df_['_5_18_medida_control_plagas'].str.strip()
df_ = df_.reset_index(drop=True)
df_

,_parent_index,_5_18_medida_control_plagas,_parent_table_name
0,1,aplicación de ceniza,Vecindarios Alimentarios - Encuesta Integrada ...
1,1,aplicación de biopreparados,Vecindarios Alimentarios - Encuesta Integrada ...
2,2,pepas aguacate rayada hervida,Vecindarios Alimentarios - Encuesta Integrada ...
3,2,agua chochos,Vecindarios Alimentarios - Encuesta Integrada ...
4,2,agua cascara de plátano,Vecindarios Alimentarios - Encuesta Integrada ...
...,...,...,...
325,893,lava los tallos de las plantas,Vecindarios Alimentarios - Encuesta Integrada ...
326,893,realiza podas,Vecindarios Alimentarios - Encuesta Integrada ...
327,894,aplica insecticidas,Vecindarios Alimentarios - Encuesta Integrada ...
328,895,usa muy poco remedios,Vecindarios Alimentarios - Encuesta Integrada ...


In [18]:
df_['_5_18_medida_control_plagas'].unique().size

180

In [19]:
# Crea la nueva columna
df_["estandarizado"] = df_["_5_18_medida_control_plagas"].copy()

# Itera sobre cada fila del dataframe
for i in range(len(df_)):
    # Compara la columna específica con todas las demás filas de la misma columna
    match = list(set(difflib.get_close_matches(df_.loc[i, "_5_18_medida_control_plagas"], df_["_5_18_medida_control_plagas"], n=23, cutoff=0.9)))
    # Reemplaza el valor en la columna específica con el valor más similar encontrado
    if match:
        df_.loc[i, "estandarizado"] = match[0]

# Elimina las filas duplicadas
df_ = df_.groupby(by=["_parent_index"], as_index=False).agg(estandar=('estandarizado', ', '.join))
df_

,_parent_index,estandar
0,1,"aplicación de cenizas, aplicación de bioprep..."
1,2,"pepas aguacate rayada hervida, agua chochos, a..."
2,3,"ceniza, agua canela, vinagre, ají con licor,"
3,4,"agua de cebolla, ají, ajo. purine ajo y ceboll..."
4,5,extractos de hierbas
...,...,...
143,891,aplicación de bioinsumos
144,892,"aplicación de bioinsumos, aplicación de agro..."
145,893,"aplicacion de paprika, control mecanico, lava ..."
146,894,aplica insecticida


In [20]:
vecindarios['_5_18_medida_control_plagas'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['estandar'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['estandar'].size != 0 else np.nan, axis=1)

### productos_control_plagas

In [21]:
reemplazar_tildes = {
    'á':'a',
    'é':'e',
    'í':'i',
    'ó':'o',
    'ú':'u'
}

In [22]:
def separar_elementos_por_comas(dataframe: pd.core.frame.DataFrame, columna:str):
    df_ = dataframe.copy()
    df_[columna] = df_[columna].str.lower().str.split(',')
    df_ = df_.explode(columna)
    df_[columna] = df_[columna].str.strip()
    df_ = df_.reset_index(drop=True)
    df_[columna] = df_[columna].replace(to_replace=reemplazar_tildes, regex=True)
    return df_

In [23]:
def estandarizar(dataframe: pd.core.frame.DataFrame, columna:str, n:int=None, cutoff:float=None):
    if not n:
        n = 50
    if not cutoff:
        cutoff = 0.90

    df_ = dataframe.copy()

    # Crea la nueva columna
    df_["estandarizado"] = df_[columna].copy()

    # Itera sobre cada fila del dataframe
    for i in range(len(df_)):
        # Compara la columna específica con todas las demás filas de la misma columna
        match = difflib.get_close_matches(df_.loc[i, columna], df_[columna], n=n, cutoff=cutoff)
        # Reemplaza el valor en la columna específica con el valor más similar encontrado
        if match:
            df_.loc[i, "estandarizado"] = match[0]

    # devolver a valores por comas
    df_ = df_.groupby(by=["_parent_index"], as_index=False).agg(estandar=('estandarizado', ', '.join))
    return df_

In [24]:
productos_control_plagas

,_5_19_producto_control_plagas,_parent_table_name,_parent_index
0,Purín de ají,Vecindarios Alimentarios - Encuesta Integrada ...,1
1,Agua de chochos,Vecindarios Alimentarios - Encuesta Integrada ...,1
2,Preparado de siero de leche,Vecindarios Alimentarios - Encuesta Integrada ...,1
3,Preparado de hoja de tomate riñón,Vecindarios Alimentarios - Encuesta Integrada ...,2
4,"Ají con licor, ceniza,",Vecindarios Alimentarios - Encuesta Integrada ...,3
...,...,...,...
259,Purín de hierbas amargas,Vecindarios Alimentarios - Encuesta Integrada ...,892
260,Caldo sulfocálcico,Vecindarios Alimentarios - Encuesta Integrada ...,892
261,no tiene maleza,Vecindarios Alimentarios - Encuesta Integrada ...,893
262,"furia , lannate",Vecindarios Alimentarios - Encuesta Integrada ...,894


In [25]:
df_ = estandarizar(separar_elementos_por_comas(productos_control_plagas, '_5_19_producto_control_plagas'), '_5_19_producto_control_plagas')
df_

,_parent_index,estandar
0,1,"purin de aji, agua de chochos, preparado de si..."
1,2,preparado de hoja de tomate riñon
2,3,"aji con licor, ceniza,"
3,4,purine de ajo
4,5,"agua de ortiga, purin de ajo, trampas,"
...,...,...
143,891,"caldo sulfocalcico, purin de hierbas amargas, ..."
144,892,"biol zapallo y zanahoria, furalancha, purin de..."
145,893,no tiene maleza
146,894,"furia, lannate"


In [26]:
vecindarios['_5_19_producto_control_plagas'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['estandar'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['estandar'].size != 0 else np.nan, axis=1)

### control_malezas

In [27]:
control_malezas

,_5_20_estrategia_control_malez,_parent_table_name,_parent_index
0,A mano,Vecindarios Alimentarios - Encuesta Integrada ...,1
1,Con herramientas,Vecindarios Alimentarios - Encuesta Integrada ...,1
2,Manual,Vecindarios Alimentarios - Encuesta Integrada ...,2
3,A mano,Vecindarios Alimentarios - Encuesta Integrada ...,3
4,A mano,Vecindarios Alimentarios - Encuesta Integrada ...,4
...,...,...,...
170,Con herramienta,Vecindarios Alimentarios - Encuesta Integrada ...,890
171,Con motoguadaña,Vecindarios Alimentarios - Encuesta Integrada ...,891
172,Con herramienta,Vecindarios Alimentarios - Encuesta Integrada ...,892
173,"herbicida, a mano",Vecindarios Alimentarios - Encuesta Integrada ...,894


In [28]:
df_ = estandarizar(separar_elementos_por_comas(control_malezas, columna='_5_20_estrategia_control_malez'), columna='_5_20_estrategia_control_malez')
df_

,_parent_index,estandar
0,1,"a mano, con herramientas"
1,2,manual
2,3,a mano
3,4,a mano
4,5,a mano
...,...,...
143,890,con herramienta
144,891,con motoguadaña
145,892,con herramienta
146,894,"herbicida, a mano"


In [29]:
vecindarios['_5_20_estrategia_control_malez'] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']]['estandar'].iloc[0] if df_.loc[df_['_parent_index']==x['_index']]['estandar'].size != 0 else np.nan, axis=1)

### oferta_alimentos

In [30]:
oferta_alimentos

,_parent_index,Producto,Unidad_Cantidad,Precio_de_Venta,Moneda,equivalencia_unidad,porcentaje_perdida_semana,comercializo_2020,motivo_no_comercializ_2020,Utilice_ese_espacio_os_campos_anteriores,_parent_table_name
0,1,Pimientos,25 fundas de 300 gr.,0.60,dolares,7.5 kg.,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
1,1,Tomate Riñón,15 fundas de 1 kilo,1,NaN,15,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
2,1,Brócoli,15 unidades de entre 300 y 400 gr.,0.50,dolares,5.25 kg,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
3,1,Lechuga,20 unidades de 200 gr.,0.50,NaN,4 kg.,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
4,1,Chochos cocinados,15 tarrina de 300 gr.,1,NaN,4.5 kg.,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
...,...,...,...,...,...,...,...,...,...,...,...
576,892,Mora,8 baldes de 13 lb,12,dolares,47.27 kg,NaN,si,No se podía circular,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
577,892,Leche,210 litros,0.47,dolares,NaN,NaN,no,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...
578,893,esquejes de pitahaya,esqueje,10,soles,kg,NaN,no,NaN,-,Vecindarios Alimentarios - Encuesta Integrada ...
579,894,girasol,1 paquete son 10 unidades,7-14,soles,kg,NaN,si,no se dedicaba a este cultivo,-,Vecindarios Alimentarios - Encuesta Integrada ...


In [31]:
oferta_alimentos['Producto'] = oferta_alimentos['Producto'].str.lower().str.strip()

In [32]:
df_1 = oferta_alimentos.copy()
df_1['Producto'] = df_1['Producto'].str.split(',')
df_1 = df_1.explode('Producto').reset_index(drop=True)
df_1['Producto'] = df_1['Producto'].str.strip()
df_1 = df_1.fillna(value='')
df_1 = df_1.replace(to_replace=reemplazar_tildes, regex=True)

In [33]:
df_ = df_1.groupby(by='_parent_index', as_index=False).agg(', '.join).replace({r'^,+':np.nan}, regex=True)
df_ = df_.fillna(value='')
df_ = df_.drop(columns=['_parent_table_name'])
df_['Moneda'] = df_['Moneda'].apply(lambda x: re.match(r'^(\w+)', x).group() if re.match(r'^(\w+)', x) else '')
df_

,_parent_index,Producto,Unidad_Cantidad,Precio_de_Venta,Moneda,equivalencia_unidad,porcentaje_perdida_semana,comercializo_2020,motivo_no_comercializ_2020,Utilice_ese_espacio_os_campos_anteriores
0,1,"pimientos, tomate riñon, brocoli, lechuga, cho...","25 fundas de 300 gr., 15 fundas de 1 kilo, 15 ...","0.60, 1, 0.50, 0.50, 1",dolares,"7.5 kg., 15, 5.25 kg, 4 kg., 4.5 kg.",,"no, no, no, no, no",,
1,2,"remolacha, lechuga, rabanos, acelga, suquini, ...","Arroba, Arroba, Arroba, Arroba, Arroba, Arroba...","Na, Na, Na, Na, Na, Na, Na",dolares,"25, 25, 25, 25, 25, 25, 25",,"no, no, no, no, no, no, no",,"Na, Na, Na, Na, Na, Na, Na"
2,3,"acelga, lechuga, rabanos, nabo chino, repollo ...",,,dolares,,,"no, no, no, no, no, no",,
3,4,"hierbas medicinales, cebolla, remolacha, apio,...","No sabe cada socia saca una parte, y se compar...",,dolares,,,"no, no, no, no, no, no, no, no",,"Mas bien se queria ampliar la produccion , Mas..."
4,5,"lechuga, brocoli, coliflor, remolacha, suquini...",,,dolares,,,"no, no, no, no, no, no, no, no, no",,
...,...,...,...,...,...,...,...,...,...,...
233,891,"aguacate, limon, jicama","15 porciones de entre 3/4 unidades, 18 mallas ...","1, 1, 1",dolares,,,"no, no, no",,
234,892,"habas, zuquini, mora, leche","2 sacos de 20 lb, 1 gaveta 15 kg., 8 baldes de...","1 dolar la 1 1/2, 8, 12, 0.47",dolares,"18.18, 15 kg, 47.27 kg,",,"si, no, si, no","No se podia circular, , No se podia circular,",
235,893,esquejes de pitahaya,esqueje,10,soles,kg,,no,,-
236,894,girasol,1 paquete son 10 unidades,7-14,soles,kg,,si,no se dedicaba a este cultivo,-


In [34]:
for i in df_.columns[1:]:
    vecindarios['_7_1_'+i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)


### produc_dejo_de_producir_vender

In [35]:
producto_dejo_producir

,otros_productos_n_comerc_2020,motivo_no_comercializo_otros,_parent_table_name,_parent_index
0,Cebolla larga,Falta de mercado,Vecindarios Alimentarios - Encuesta Integrada ...,1
1,NaN,Na,Vecindarios Alimentarios - Encuesta Integrada ...,2
2,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...,3
3,No aplica,NaN,Vecindarios Alimentarios - Encuesta Integrada ...,4
4,NaN,NaN,Vecindarios Alimentarios - Encuesta Integrada ...,5
...,...,...,...,...
169,Hortalizas varias,Se siembra menos porque no hay garantía de venta,Vecindarios Alimentarios - Encuesta Integrada ...,891
170,Papas,Porque llegó la enfermedad de aparatiosis,Vecindarios Alimentarios - Encuesta Integrada ...,892
171,-,-,Vecindarios Alimentarios - Encuesta Integrada ...,893
172,rosas,-,Vecindarios Alimentarios - Encuesta Integrada ...,894


In [36]:
producto_dejo_producir['otros_productos_n_comerc_2020'] = producto_dejo_producir['otros_productos_n_comerc_2020'].str.lower().str.strip()
producto_dejo_producir['motivo_no_comercializo_otros'] = producto_dejo_producir['motivo_no_comercializo_otros'].str.lower().str.strip()
producto_dejo_producir = producto_dejo_producir.drop(columns=['_parent_table_name'])

In [37]:
df_1 = producto_dejo_producir.copy()
df_1['otros_productos_n_comerc_2020'] = df_1['otros_productos_n_comerc_2020'].str.split(',')
df_1 = df_1.explode('otros_productos_n_comerc_2020').reset_index(drop=True)
df_1['otros_productos_n_comerc_2020'] = df_1['otros_productos_n_comerc_2020'].str.strip()
df_1['motivo_no_comercializo_otros'] = df_1['motivo_no_comercializo_otros'].str.strip()
df_1 = df_1.fillna(value='')
df_1 = df_1.replace(to_replace=reemplazar_tildes, regex=True)

In [38]:
df_ = df_1.groupby(by='_parent_index', as_index=False).agg(', '.join).replace({r'^,+':np.nan}, regex=True)
df_ = df_.fillna(value='')
df_ = df_.replace({'-':''}, regex=True)
df_

,_parent_index,otros_productos_n_comerc_2020,motivo_no_comercializo_otros
0,1,cebolla larga,falta de mercado
1,2,,na
2,3,,
3,4,no aplica,
4,5,,
...,...,...,...
157,891,hortalizas varias,se siembra menos porque no hay garantia de venta
158,892,papas,porque llego la enfermedad de aparatiosis
159,893,,
160,894,rosas,


In [39]:
for i in df_.columns[1:]:
    vecindarios['_7_2_'+i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### productos_vende_canal

In [40]:
producto_vende_canal

,_parent_index,producto_vendido_canal,nombre_producto_vende,_index,_parent_table_name
0,1,hortalizas___verduras__tubérculos,Pimientos,1,Vecindarios Alimentarios - Encuesta Integrada ...
1,1,hortalizas___verduras__tubérculos,Tomate Riñón,2,Vecindarios Alimentarios - Encuesta Integrada ...
2,1,hortalizas___verduras__tubérculos,Brócoli,3,Vecindarios Alimentarios - Encuesta Integrada ...
3,1,arroz__fideos_y_granos___menestras,Chochos cocinados,4,Vecindarios Alimentarios - Encuesta Integrada ...
4,2,otro_producto,Na,5,Vecindarios Alimentarios - Encuesta Integrada ...
...,...,...,...,...,...
323,892,hortalizas___verduras__tubérculos,Hierbas medicinales,324,Vecindarios Alimentarios - Encuesta Integrada ...
324,893,frutas,esquejes de pitahaya,325,Vecindarios Alimentarios - Encuesta Integrada ...
325,893,frutas,pitahaya,326,Vecindarios Alimentarios - Encuesta Integrada ...
326,894,otro_producto,Girasol,327,Vecindarios Alimentarios - Encuesta Integrada ...


In [41]:
canales_venta_productores

,_index,local_venta_productores,nombre,motivos,transporte_vende_productos,distancia,canal_principal_2019,canal_principal_2020,_parent_table_name,_parent_index
0,1,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializó o intercambio,Productos_Vende_Canal,1
1,2,bioferias,Feria Casa Mitómana,Porque está disponibles para los agricultores,Bus y taxi,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio,Productos_Vende_Canal,1
2,3,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio,Productos_Vende_Canal,2
3,4,bioferias,Feria Casa Mitómana,Porque está disponibles para los agricultores,Bus y taxi,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio,Productos_Vende_Canal,2
4,5,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio,Productos_Vende_Canal,3
...,...,...,...,...,...,...,...,...,...,...
400,401,otro_canal,Mercado Mayorista de Ambato,Porque hay demanda,Camioneta propia,de_5_a_10_km,Mercado Mayorista de Ambato,Mercado Mayorista de Ambato,Productos_Vende_Canal,324
401,402,otro_canal,"huerto, facebook",facilidad de venta,-,NaN,-,-,Productos_Vende_Canal,325
402,403,otro_canal,consumo propio,-,-,NaN,-,-,Productos_Vende_Canal,326
403,404,otro_canal,Mercado Santa Rosa,venta de flores,camioneta,de_5_a_10_km,no sembraba girasoles,no sembraba girasoles,Productos_Vende_Canal,327


In [42]:
df_ = pd.merge(left=producto_vende_canal, right=canales_venta_productores, left_on='_index', right_on='_parent_index', suffixes=('','_y'))
df_ = df_.drop(columns=[i for i in df_.columns if i.endswith('_y')])
df_ = df_.fillna(value='')
df_ = df_.drop_duplicates(subset=['_parent_index','_index','nombre_producto_vende'])
df_

,_parent_index,producto_vendido_canal,nombre_producto_vende,_index,_parent_table_name,local_venta_productores,nombre,motivos,transporte_vende_productos,distancia,canal_principal_2019,canal_principal_2020
0,1,hortalizas___verduras__tubérculos,Pimientos,1,Vecindarios Alimentarios - Encuesta Integrada ...,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializó o intercambio
2,1,hortalizas___verduras__tubérculos,Tomate Riñón,2,Vecindarios Alimentarios - Encuesta Integrada ...,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio
4,1,hortalizas___verduras__tubérculos,Brócoli,3,Vecindarios Alimentarios - Encuesta Integrada ...,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio
6,1,arroz__fideos_y_granos___menestras,Chochos cocinados,4,Vecindarios Alimentarios - Encuesta Integrada ...,bioferias,Feria La Carolina,Porque está disponibles para los agricultores,Camioneta de alquiler,más_de_10_km,Feria La Carolina,No se comercializaba o intercambio
8,2,otro_producto,Na,5,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,Na,Na,Na,no_aplica,Na,Na
...,...,...,...,...,...,...,...,...,...,...,...,...
400,892,hortalizas___verduras__tubérculos,Hierbas medicinales,324,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,Mercado Mayorista de Ambato,Porque hay demanda,Camioneta propia,de_5_a_10_km,Mercado Mayorista de Ambato,Mercado Mayorista de Ambato
401,893,frutas,esquejes de pitahaya,325,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,"huerto, facebook",facilidad de venta,-,,-,-
402,893,frutas,pitahaya,326,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,consumo propio,-,-,,-,-
403,894,otro_producto,Girasol,327,Vecindarios Alimentarios - Encuesta Integrada ...,otro_canal,Mercado Santa Rosa,venta de flores,camioneta,de_5_a_10_km,no sembraba girasoles,no sembraba girasoles


In [43]:
df_['producto_vendido_canal'] = df_['producto_vendido_canal'].str.lower().str.strip()
df_['nombre_producto_vende'] = df_['nombre_producto_vende'].str.lower().str.strip()
df_['local_venta_productores'] = df_['local_venta_productores'].str.lower().str.strip()
df_['nombre'] = df_['nombre'].str.lower().str.strip()
df_['motivos'] = df_['motivos'].str.lower().str.strip()
df_['transporte_vende_productos'] = df_['transporte_vende_productos'].str.lower().str.strip()
df_['distancia'] = df_['distancia'].str.lower().str.strip()
df_['canal_principal_2019'] = df_['canal_principal_2019'].str.lower().str.strip()
df_['canal_principal_2020'] = df_['canal_principal_2020'].str.lower().str.strip()

In [44]:
df_ = df_.replace(to_replace=reemplazar_tildes|{'-':''}, regex=True)

In [45]:
df__ = df_.groupby(by='_parent_index', as_index=False).agg(', '.join).replace({r'^,+':np.nan}, regex=True).fillna(value='')
df__

c:\Users\norte\anaconda3\envs\hackspace_portal\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,_parent_index,producto_vendido_canal,nombre_producto_vende,_parent_table_name,local_venta_productores,nombre,motivos,transporte_vende_productos,distancia,canal_principal_2019,canal_principal_2020
0,1,"hortalizas___verduras__tuberculos, hortalizas_...","pimientos, tomate riñon, brocoli, chochos coci...",Vecindarios Alimentarios Encuesta Integrada (...,"bioferias, bioferias, bioferias, bioferias","feria la carolina, feria la carolina, feria la...","porque esta disponibles para los agricultores,...","camioneta de alquiler, camioneta de alquiler, ...","mas_de_10_km, mas_de_10_km, mas_de_10_km, mas_...","feria la carolina, feria la carolina, feria la...","no se comercializo o intercambio, no se comerc..."
1,2,otro_producto,na,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,na,na,na,no_aplica,na,na
2,3,otro_producto,,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,,,,no_aplica,,
3,4,otro_producto,,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,,,,no_aplica,,
4,5,otro_producto,,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,,,,no_aplica,,
...,...,...,...,...,...,...,...,...,...,...,...
142,891,"frutas, frutas, hortalizas___verduras__tuberculos","aguacate, limon, verduras y tuberculos variados",Vecindarios Alimentarios Encuesta Integrada (...,"bioferias, bioferias, bioferias","feria de carcelen, feria de carcelen, feria de...","porque esta disponible, porque esta disponible...","bus y/o camioneta alquilada, bus y/o camioneta...","mas_de_10_km, mas_de_10_km, mas_de_10_km","feria de carcelen, feria de carcelen, feria de...","no se comercializaba, no se comercializaba, no..."
143,892,"leche_y_productos_lacteos, hortalizas___verdur...","leche, zuquini, habas, hortalizas variadas, hi...",Vecindarios Alimentarios Encuesta Integrada (...,"otro_canal, bioferias, otro_canal, bioferias, ...","empresa indulac, mercado madre tierra, mercado...","porque hay demanda, porque esta disponible y s...","recoleccion a pie de finca, camioneta propia, ...","no_aplica, mas_de_10_km, de_5_a_10_km, mas_de_...","intermediario, mercado madre tierra, mercado m...","intermediario, no se comercializo, no se comer..."
144,893,"frutas, frutas","esquejes de pitahaya, pitahaya",Vecindarios Alimentarios Encuesta Integrada (...,"otro_canal, otro_canal","huerto, facebook, consumo propio","facilidad de venta,",,,,
145,894,otro_producto,girasol,Vecindarios Alimentarios Encuesta Integrada (...,otro_canal,mercado santa rosa,venta de flores,camioneta,de_5_a_10_km,no sembraba girasoles,no sembraba girasoles


In [46]:
df_1 = df__['producto_vendido_canal'].str.get_dummies(sep=', ')
df_1.columns = ['producto_vendido_canal/'+i for i in df_1.columns]
df_2 = df__['local_venta_productores'].str.get_dummies(sep=', ')
df_2.columns = ['local_venta_productores/'+i for i in df_2.columns]
df_3 = df__['distancia'].str.get_dummies(sep=', ')
df_3.columns = ['distancia/'+i for i in df_3.columns]
df__ = df__.drop(columns=['producto_vendido_canal', 'local_venta_productores', 'distancia'])
df__ = pd.concat([df__, df_1, df_2, df_3], axis=1)
df__

,_parent_index,nombre_producto_vende,_parent_table_name,nombre,motivos,transporte_vende_productos,canal_principal_2019,canal_principal_2020,producto_vendido_canal/arroz__fideos_y_granos___menestras,producto_vendido_canal/carnes_y_pescados,...,local_venta_productores/intermediarios,local_venta_productores/mayoristas,local_venta_productores/mercados_de_abasto,local_venta_productores/otro_canal,local_venta_productores/tiendas_especializadas__organicos_o_agroecologicos_,distancia/de_1_a_5_km,distancia/de_5_a_10_km,distancia/mas_de_10_km,distancia/menor_a_1_km,distancia/no_aplica
0,1,"pimientos, tomate riñon, brocoli, chochos coci...",Vecindarios Alimentarios Encuesta Integrada (...,"feria la carolina, feria la carolina, feria la...","porque esta disponibles para los agricultores,...","camioneta de alquiler, camioneta de alquiler, ...","feria la carolina, feria la carolina, feria la...","no se comercializo o intercambio, no se comerc...",1,0,...,0,0,0,0,0,0,0,1,0,0
1,2,na,Vecindarios Alimentarios Encuesta Integrada (...,na,na,na,na,na,0,0,...,0,0,0,1,0,0,0,0,0,1
2,3,,Vecindarios Alimentarios Encuesta Integrada (...,,,,,,0,0,...,0,0,0,1,0,0,0,0,0,1
3,4,,Vecindarios Alimentarios Encuesta Integrada (...,,,,,,0,0,...,0,0,0,1,0,0,0,0,0,1
4,5,,Vecindarios Alimentarios Encuesta Integrada (...,,,,,,0,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,891,"aguacate, limon, verduras y tuberculos variados",Vecindarios Alimentarios Encuesta Integrada (...,"feria de carcelen, feria de carcelen, feria de...","porque esta disponible, porque esta disponible...","bus y/o camioneta alquilada, bus y/o camioneta...","feria de carcelen, feria de carcelen, feria de...","no se comercializaba, no se comercializaba, no...",0,0,...,0,0,0,0,0,0,0,1,0,0
143,892,"leche, zuquini, habas, hortalizas variadas, hi...",Vecindarios Alimentarios Encuesta Integrada (...,"empresa indulac, mercado madre tierra, mercado...","porque hay demanda, porque esta disponible y s...","recoleccion a pie de finca, camioneta propia, ...","intermediario, mercado madre tierra, mercado m...","intermediario, no se comercializo, no se comer...",1,0,...,0,0,0,1,0,0,1,1,0,1
144,893,"esquejes de pitahaya, pitahaya",Vecindarios Alimentarios Encuesta Integrada (...,"huerto, facebook, consumo propio","facilidad de venta,",,,,0,0,...,0,0,0,1,0,0,0,0,0,0
145,894,girasol,Vecindarios Alimentarios Encuesta Integrada (...,mercado santa rosa,venta de flores,camioneta,no sembraba girasoles,no sembraba girasoles,0,0,...,0,0,0,1,0,0,1,0,0,0


In [47]:
df_ = df__.copy()

In [48]:
df_.drop(columns=['_parent_table_name'], inplace=True)

In [49]:
for i in df_.columns[1:]:
    vecindarios['_Productos_canal_venta_'+i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### aplicacion_biopreparados

In [50]:
aplicacion_biopreparados = aplicacion_biopreparados.fillna(value='')
aplicacion_biopreparados = aplicacion_biopreparados.drop(columns=['_parent_table_name'])
aplicacion_biopreparados

,_10_2_Quien_aplica_biopreparad,sexo_contratado,parentesco_familiar,Indique_otro_pariente,otra_persona_aplica_bioprep,_10_2_1_usa_proteccion,_parent_index
0,yo_mismo,,,,,no,1
1,pariente_s,,esposos,,,no,1
2,yo_mismo,,,,,no,11
3,pariente_s,,hijos_hijastros,,,no,11
4,pariente_s,,hijos_hijastros,,,parcial,12
...,...,...,...,...,...,...,...
123,yo_mismo,,,,,no,891
124,pariente_s,,esposos,,,parcial,892
125,yo_mismo,,,,,parcial,893
126,yo_mismo,,,,,no,894


In [51]:
columnas = ['_10_2_Quien_aplica_biopreparad','sexo_contratado','parentesco_familiar','Indique_otro_pariente','otra_persona_aplica_bioprep','_10_2_1_usa_proteccion']

In [52]:
df_ = aplicacion_biopreparados.copy()
df_ = pd.get_dummies(df_, columns=columnas, prefix=columnas, prefix_sep='/')
df_

,_parent_index,_10_2_Quien_aplica_biopreparad/contratado_s,_10_2_Quien_aplica_biopreparad/otra_persona,_10_2_Quien_aplica_biopreparad/pariente_s,_10_2_Quien_aplica_biopreparad/yo_mismo,sexo_contratado/,sexo_contratado/hombre,sexo_contratado/mujer,parentesco_familiar/,parentesco_familiar/esposos,...,parentesco_familiar/padres,parentesco_familiar/yerno_nuera,Indique_otro_pariente/,Indique_otro_pariente/cuñado,otra_persona_aplica_bioprep/,otra_persona_aplica_bioprep/Hijo,_10_2_1_usa_proteccion/,_10_2_1_usa_proteccion/no,_10_2_1_usa_proteccion/parcial,_10_2_1_usa_proteccion/si
0,1,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
1,1,0,0,1,0,1,0,0,0,1,...,0,0,1,0,1,0,0,1,0,0
2,11,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
3,11,0,0,1,0,1,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
4,12,0,0,1,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,891,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0
124,892,0,0,1,0,1,0,0,0,1,...,0,0,1,0,1,0,0,0,1,0
125,893,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,0,1,0
126,894,0,0,0,1,1,0,0,1,0,...,0,0,1,0,1,0,0,1,0,0


In [53]:
for i in df_.columns[1:]:
    vecindarios['_aplicación_biopreparados_'+i] = vecindarios.apply(lambda x: df_.loc[df_['_parent_index']==x['_index']][i].iloc[0] if df_.loc[df_['_parent_index']==x['_index']][i].size != 0 else np.nan, axis=1)

### personas_trabajan_puesto_de_venta

In [71]:
df_ = personas_trabajan_puesto_venta.copy()
df_.drop(columns=['_parent_table_name','otro_pariente_empleado','otra_persona_empleada',*[i for i in df_.columns if i.startswith('Indica')]], inplace=True)
df_['_b_horas_porsemana_trabaja'] = df_['_b_horas_porsemana_trabaja'].fillna(value='')
df_['_b_horas_porsemana_trabaja'] = df_['_b_horas_porsemana_trabaja'].apply(lambda x: re.match(r'\d+', x).group() if re.match(r'\d+', x) else 'esporadico')
df_

,personas_empleadas_puesto,pariente_empleado,_b_horas_porsemana_trabaja,_c_persona_recibe_remuneracion,_parent_index
0,pariente,esposos,8,no,24
1,empleado,NaN,7,no,48
2,pariente,esposos,8,no,50
3,pariente,esposos,8,si,53
4,pariente,esposos,8,no,55
5,pariente,esposos,8,si,56
6,pariente,hijos_hijastros,esporadico,no,74
7,pariente,hijos_hijastros,7,si,107
8,empleado,NaN,10,si,114
9,empleado,NaN,10,si,114


In [73]:
columns = ['personas_empleadas_puesto','pariente_empleado','_b_horas_porsemana_trabaja','_c_persona_recibe_remuneracion']

In [74]:
pd.get_dummies(df_, columns=columns, prefix=columns, prefix_sep='/')

,_parent_index,personas_empleadas_puesto/empleado,personas_empleadas_puesto/pariente,pariente_empleado/esposos,pariente_empleado/hermanos,pariente_empleado/hijos_hijastros,pariente_empleado/padres,_b_horas_porsemana_trabaja/10,_b_horas_porsemana_trabaja/18,_b_horas_porsemana_trabaja/42,_b_horas_porsemana_trabaja/5,_b_horas_porsemana_trabaja/60,_b_horas_porsemana_trabaja/7,_b_horas_porsemana_trabaja/70,_b_horas_porsemana_trabaja/8,_b_horas_porsemana_trabaja/esporadico,_c_persona_recibe_remuneracion/no,_c_persona_recibe_remuneracion/si
0,24,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,48,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
2,50,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
3,53,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
4,55,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0
5,56,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
6,74,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
7,107,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
8,114,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
9,114,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [7]:
vecindarios_productor = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='productor'].reset_index(drop=True)

In [8]:
vecindarios_comerciante = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='comerciante'].reset_index(drop=True)

In [9]:
vecindarios_consumidor = vecindarios.loc[vecindarios['_1_5_perfil_entrevistado']=='consumidor'].reset_index(drop=True)

In [56]:
df = pd.concat([df_productor, df_comerciante, df_consumidor], axis=0)
df

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,huerto_tipo_relacion/interc_produccion,huerto_tipo_relacion/interc_insumos,huerto_tipo_relacion/acopio_comercio,huerto_tipo_relacion/ensenanza,huerto_tipo_relacion/otra_relacion,huerto_otra_relacion,_2019_huerto_relacion,_2022_huerto_relacion,huertos_observaciones,_parent_index_y
0,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2022-07-29 22:39:16.546,2022-07-29 23:12:08.197999,1709974719,2022-07-29,Gabriela Paredes,quito,fuera,Tolontag Centro,productor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3864,818,2022-10-06 17:55:56.364,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3865,818,2022-10-06 17:55:56.364,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3866,818,2022-10-06 17:55:56.364,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3867,818,2022-10-06 17:55:56.364,2022-10-06 19:07:10.961001,01- colegio ardiles,2022-07-27,Melany Arango Lopez,lima,pachacamac,NaN,consumidor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df.to_csv('primera_version_raw.csv', index=False)